# Data Handling and Initial Analyses

In [ ]:
#Imports

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from helpers import load_data_with_columns, date_to_int

In [ ]:
#Loading the data

DATAFOLDER = '../data/'
character_metadata = load_data_with_columns(DATAFOLDER, 'character.metadata.tsv')
movie_metadata = load_data_with_columns(DATAFOLDER, 'movie.metadata.tsv')

movie_metadata = date_to_int(movie_metadata, 'movie_release_date')

character_metadata = date_to_int(character_metadata, 'movie_release_date')